## Score the predicted Hi-C matrices

In [ ]:
import os

import numpy as np
import pandas as pd
from hicrep import hicrepSCC
from hicexplorer import hicFindTADs
import pygenometracks
import matplotlib.pyplot as plt
import cooler
import traceback
from sklearn import metrics as metrics
from scipy import sparse
from hicrep.utils import readMcool



In [ ]:
ROOT_PATH = '/Users/wolffjoa/data_local/hicgan/hyperparameter'

ROOT_PATH_25KB = os.path.join(ROOT_PATH, 'hyperparameter_paper_25kb/')
ROOT_PATH_25KB_predicted = os.path.join(ROOT_PATH_25KB, "predicted")
hic_matrix_25kb_path = os.path.join(ROOT_PATH_25KB, 'GM12878_25kb_cooler_coarsen_chr1.cool')
files_25kb = os.listdir(ROOT_PATH_25KB_predicted)



### helper functions

In [ ]:
def getMatrixFromCooler(pCoolerFilePath, pChromNameStr):
    #returns sparse csr matrix from cooler file for given chromosome name
    sparseMatrix = None
    binSizeInt = 0
    try:
        coolerMatrix = cooler.Cooler(pCoolerFilePath)
        sparseMatrix = coolerMatrix.matrix(sparse=True,balance=False).fetch(pChromNameStr)
        sparseMatrix = sparseMatrix.tocsr() #so it can be sliced later
        binSizeInt = int(coolerMatrix.binsize)
    except Exception as e:
        traceback.print_exc()
        print(coolerMatrix.chromnames)
        print(e)
    return sparseMatrix, binSizeInt

def computePearsonCorrelation(pCoolerFile1, pCoolerFile2, 
                              pWindowsize_bp,
                              pModelChromList, pTargetChromStr,
                              pModelCellLineList, pTargetCellLineStr,
                              pPlotOutputFile=None, pCsvOutputFile=None):
    '''
    compute distance-stratified pearson correlation for target chromosome
    directly from cooler files and plot or write to file

    Parameters:
        pCoolerFile1 (str): Path to cooler file 1
        pCoolerFile2 (str): Path to cooler file 2
        pWindowsize_bp (int): Windowsize in basepairs for which correlations shall be computed
        pModelChromList (list): List of strings, will appear in plot title
        pModelCellLineList (list): List of strings, will appear in plot title
        pTargetChromStr (str): the target chromosome, e.g. >chr10< or >10<
        pTargetCellLineStr (str): the target cell line, will appear in plot title
        pPlotOutputFile (str): filename of correlation plot
        pCsvOutputFile (str): filename of correlation csv file
    
    Returns:
        None
    ''' 

    sparseMatrix1, binsize1 = getMatrixFromCooler(pCoolerFile1, pTargetChromStr)
    sparseMatrix2, binsize2 = getMatrixFromCooler(pCoolerFile2, pTargetChromStr)
    errorMsg = ""
    if sparseMatrix1 is None:
        errorMsg += "Chrom {:s} could not be loaded from {:s}\n"
        errorMsg = errorMsg.format(str(pTargetChromStr), pCoolerFile1)
    if sparseMatrix2 is None:
        errorMsg += "Chrom {:s} could not be loaded from {:s}\n"
        errorMsg = errorMsg.format(str(pTargetChromStr), pCoolerFile2)
    if errorMsg != "":
        errorMsg += "Potential reasons: Wrong file format, wrong chromosome naming scheme or chromosome missing"
        raise SystemExit(errorMsg)
    if binsize1 != binsize2:
        errorMsg = "Aborting. Binsizes of matrices are not equal\n"
        errorMsg += "{:s} -- {:d}bp\n"
        errorMsg += "{:s} -- {:d}bp\n"
        errorMsg = errorMsg.format(pCoolerFile1,binsize1, pCoolerFile2, binsize2)
        raise SystemExit(errorMsg)
    resultsDf = computePearsonCorrelationSparse(pSparseCsrMatrix1= sparseMatrix1,
                                                pSparseCsrMatrix2= sparseMatrix2,
                                                pBinsize= binsize1,
                                                pWindowsize_bp= pWindowsize_bp,
                                                pModelChromList= pModelChromList,
                                                pTargetChromStr= pTargetChromStr,
                                                pModelCellLineList= pModelCellLineList,
                                                pTargetCellLineStr= pTargetCellLineStr)
    if pCsvOutputFile is not None:
        resultsDf.to_csv(pCsvOutputFile)
    if pPlotOutputFile is not None:
        plotPearsonCorrelationDf(pResultsDfList=[resultsDf], 
                                 pLegendList=["Pearson corr."],
                                 pOutfile=pPlotOutputFile,
                                 pMethod="pearson")
    return resultsDf

def maskFunc(pArray, pWindowSize=0):
    #mask a trapezoid along the (main) diagonal of a 2D array
    #this code is copied from the study project by Ralf Krauth
    #https://github.com/MasterprojectRK/HiCPrediction/blob/master/hicprediction/createTrainingSet.py
    maskArray = np.zeros(pArray.shape)
    upperTriaInd = np.triu_indices(maskArray.shape[0]) # pylint: disable=unsubscriptable-object
    notRequiredTriaInd = np.triu_indices(maskArray.shape[0], k=pWindowSize) # pylint: disable=unsubscriptable-object
    maskArray[upperTriaInd] = 1
    maskArray[notRequiredTriaInd] = 0
    return maskArray

def computePearsonCorrelationSparse(pSparseCsrMatrix1, pSparseCsrMatrix2, 
                                    pBinsize, pWindowsize_bp, 
                                    pModelChromList, pTargetChromStr, 
                                    pModelCellLineList, pTargetCellLineStr):
    '''
    compute distance-stratified Pearson correlation from two sparse matrices

    Parameters:
        pSparseCsrMatrix1 (scipy.sparse.csr_matrix): sparse csr matrix 1
        pSparseCsrMatrix2 (scipy.sparse.csr_matrix): sparse csr matrix 2
        pBinsize (int): the binsize of each bin in the sparse matrices
        pWindowsize_bp (int): the windowsize in basepairs for which correlations shall be computed
        pModelChromList (list): list of strings, will appear in plot title
        pTargetChromStr (str): the target chromosome, e.g. >chr10< or >10<
        pTargetCellLineStr (str): the target cell line, will appear in plot title
        pModelCellLineList (list): List of strings, will appear in plot title

    Returns:
        (pandas.DataFrame): Pandas dataframe containing the correlations per distance 
    '''
    numberOfDiagonals = int(np.round(pWindowsize_bp/pBinsize))
    if numberOfDiagonals < 1:
        msg = "Window size must be larger than bin size of matrices.\n"
        msg += "Remember to specify window in basepairs, not bins."
        raise SystemExit(msg)
    shape1 = pSparseCsrMatrix1.shape
    shape2 = pSparseCsrMatrix2.shape
    if shape1 != shape2:
        msg = "Aborting. Shapes of matrices are not equal.\n"
        msg += "Shape 1: ({:d},{:d}); Shape 2: ({:d},{:d})"
        msg = msg.format(shape1[0],shape1[1],shape2[0],shape2[1])
        raise SystemExit(msg)
    if numberOfDiagonals > shape1[0]-1:
        msg = "Aborting. Window size {0:d} larger than matrix size {:d}"
        msg = msg.format(numberOfDiagonals, shape1[0]-1)
        raise SystemExit(msg)
    
    trapezIndices = np.mask_indices(shape1[0],maskFunc,k=numberOfDiagonals)
    reads1 = np.array(pSparseCsrMatrix1[trapezIndices])[0]
    reads2 = np.array(pSparseCsrMatrix2[trapezIndices])[0]

    matrixDf = pd.DataFrame(columns=['first','second','distance','reads1','reads2'])
    matrixDf['first'] = np.uint32(trapezIndices[0])
    matrixDf['second'] = np.uint32(trapezIndices[1])
    matrixDf['distance'] = np.uint32(matrixDf['second'] - matrixDf['first'])
    matrixDf['reads1'] = np.float32(reads1)
    matrixDf['reads2'] = np.float32(reads2)
    matrixDf.fillna(0, inplace=True)

    pearsonAucIndices, pearsonAucValues = getCorrelation(matrixDf,'distance', 'reads1', 'reads2', 'pearson')
    pearsonAucScore = metrics.auc(pearsonAucIndices, pearsonAucValues)
    spearmanAucIncides, spearmanAucValues = getCorrelation(matrixDf,'distance', 'reads1', 'reads2', 'spearman')
    spearmanAucScore = metrics.auc(spearmanAucIncides, spearmanAucValues)
    # print("PearsonAUC: {:.3f}".format(pearsonAucScore))
    # print("SpearmanAUC: {:.3f}".format(spearmanAucScore))

    columns = ["corrMeth", "modelChroms", "targetChrom", 
                           "modelCellLines", "targetCellLine", 
                           "R2", "MSE", "MAE", "MSLE", "AUC",
                           "binsize", "windowsize"]
    columns.extend(sorted(list(matrixDf.distance.unique())))
    resultsDf = pd.DataFrame(columns=columns)
    resultsDf["corrMeth"] = ["pearson", "spearman"]
    resultsDf.set_index("corrMeth", inplace=True)
    resultsDf.loc[:, 'modelChroms'] = ", ".join([str(x) for x in pModelChromList])
    resultsDf.loc[:, 'targetChrom'] = pTargetChromStr
    resultsDf.loc[:, 'modelCellLines'] = ", ".join([str(x) for x in pModelCellLineList])
    resultsDf.loc[:, 'targetCellLine'] = pTargetCellLineStr
    resultsDf.loc[:, "R2"] = metrics.r2_score(matrixDf['reads2'], matrixDf['reads1'])
    resultsDf.loc[:, 'MSE'] = metrics.mean_squared_error( matrixDf['reads2'], matrixDf['reads1'])
    resultsDf.loc[:, 'MAE'] = metrics.mean_absolute_error( matrixDf['reads2'], matrixDf['reads1'])
    resultsDf.loc[:, 'MSLE'] = metrics.mean_squared_log_error(matrixDf['reads2'], matrixDf['reads1'])
    resultsDf.loc['pearson', 'AUC'] = pearsonAucScore 
    resultsDf.loc['spearman', 'AUC'] = spearmanAucScore
    resultsDf.loc[:, 'binsize'] = pBinsize
    resultsDf.loc[:, 'windowsize'] = pWindowsize_bp
    
    for pearsonIndex, corrValue in zip(pearsonAucIndices,pearsonAucValues):
        columnName = int(round(pearsonIndex * matrixDf.distance.max()))
        resultsDf.loc["pearson", columnName] = corrValue
    for spearmanIndex, corrValue in zip(spearmanAucIncides,spearmanAucValues):
        columnName = int(round(spearmanIndex * matrixDf.distance.max()))
        resultsDf.loc["spearman", columnName] = corrValue
    return resultsDf

  
def plotPearsonCorrelationDf(pResultsDfList, pLegendList, pOutfile, pMethod="pearson"):
    #helper function to plot distance-stratified Pearson correlation stored in pandas dataframes
    if pMethod not in ["pearson", "spearman"]:
        print("plotting only supported for 'pearson' and 'spearman' correlation methods")
        return
    if pResultsDfList is None or pLegendList is None:
        return
    if not isinstance(pResultsDfList,list) or not isinstance(pLegendList,list):
        return
    legendStrList = [str(x) for x in pLegendList]
    if len(pResultsDfList) != len(legendStrList):
        msg = "can't plot, too many / too few legends\n"
        msg += "no. of legend entries should be: {:d}, given {:d}"
        msg = msg.format(len(pResultsDfList), len(legendStrList))
        print(msg)
        return
    
    fig1, ax1 = plt.subplots()
    ax1.set_ylabel("{:s} correlation".format(pMethod[0].upper() + pMethod[1:] ))
    ax1.set_xlabel("Genomic distance / Mbp")
    trainChromSet = set()
    targetChromSet = set()
    trainCellLineSet = set()
    targetCellLineSet = set()
    maxXVal = 0
    for i, resultsDf in enumerate(pResultsDfList):
        try:
            resolutionInt = int(resultsDf.loc[pMethod, 'binsize'])
            windowsize_bp = int(resultsDf.loc[pMethod, 'windowsize'])
            trainChromSet.add(resultsDf.loc[pMethod, 'modelChroms'])
            targetChromSet.add(resultsDf.loc[pMethod, 'targetChrom'])
            trainCellLineSet.add(resultsDf.loc[pMethod, 'modelCellLines'])
            targetCellLineSet.add(resultsDf.loc[pMethod, 'targetCellLine'])
            area_under_corr_curve = resultsDf.loc[pMethod, 'AUC']
            maxDist_bp = int(windowsize_bp / resolutionInt)
            columnNameList = [x for x in range(maxDist_bp)]
            corrXValues = np.arange(maxDist_bp) * resolutionInt / 1000000
            corrYValues = resultsDf.loc[pMethod, columnNameList].values.astype("float32")
        except Exception as e:
            msg = str(e) + "\n"
            msg += "results dataframe {:d} does not contain all relevant fields (binsize, distance stratified pearson correlation data etc.)"
            msg = msg.format(i)
            print(msg)
        label = pLegendList[i]
        if label is None:
            label = pMethod + " / AUC: {:.3f}".format(area_under_corr_curve)
        else:
            label = label + " / AUC: {:.3f}".format(area_under_corr_curve)
        ax1.plot(corrXValues, corrYValues, label = label)
        maxXVal = max(maxXVal, corrXValues[-1])
    titleStr = "Pearson correlation vs. genomic distance"
    if len(trainChromSet) == len(targetChromSet) == len(trainCellLineSet) == len(targetCellLineSet) == 1:
        titleStr += "\n {:s}, {:s} on {:s}, {:s}"
        titleStr = titleStr.format(list(trainCellLineSet)[0], list(trainChromSet)[0], list(targetCellLineSet)[0], list(targetChromSet)[0])
    ax1.set_title(titleStr)
    ax1.set_ylim([0,1])
    ax1.set_xlim([0,maxXVal])
    ax1.grid(True)
    ax1.legend(frameon=False, loc="upper right")
    
    if pOutfile is None:
        outfile = "correlation.png"
        fig1.savefig(outfile)
    else:
        outfile = pOutfile
        if os.path.splitext(outfile)[1] not in ['.png', '.svg', '.pdf']:
            outfile = os.path.splitext(pOutfile)[0] + '.png'
            msg = "Outfile must have png, pdf or svg file extension.\n"
            msg += "Renamed outfile to {:s}".format(outfile)
            print(msg)
        fig1.savefig(outfile)
    plt.close(fig1)
    del fig1, ax1

def getCorrelation(pData, pDistanceField, pTargetField, pPredictionField, pCorrMethod):
    """
    Helper method to calculate correlation
    This method has originally been written by Andre Bajorat during his study project,
    licensed under the MIT License: 
    https://github.com/abajorat/HiCPrediction/blob/master/hicprediction/predict.py
    It has been adapted by Ralf Krauth during his study project:
    https://github.com/MasterprojectRK/HiCPrediction/blob/master/hicprediction/predict.py
    
    Parameters:
        pData (pandas.DataFrame): Pandas dataframe with read counts / distances
        pDistanceField (str): the column name of the distance Field in the dataframe
        pTargetField (str): the column name of the target read counts in the dataframe
        pPredictionField (str): column name of the predicted read counts in the dataframe
        pCorrMethod (str): any of the correlation methods supported by pandas DataFrame corr method
    
    Returns:
        indices (list): integer list of index values 
        values (list): float list of correlation values
    """
    # Step 1: Calculate the Correlation Matrix
    correlation_matrix = pData.groupby(pDistanceField, group_keys=False)[[pTargetField, pPredictionField]].corr(method=pCorrMethod)

    # Step 2: Extracting Relevant Correlation Values
    correlation_values = correlation_matrix.iloc[0::2, -1].reset_index(drop=True)

    # Step 3: Handling NaNs
    correlation_values.dropna(inplace=True)

    # Step 4: Preparing the Output
    indices = correlation_values.index.tolist()
    indices = np.array(indices, dtype=np.int32)
    max_distance = pData[pDistanceField].max()
    indices = indices / max_distance

    values = correlation_values.values

    # Return the final indices and values
    return indices, values

In [ ]:
def scoreAndPlot(correlationMethodList, errorType, outputFolder, dataFolder, tadFolder, matrixOutputName, originalDataMatrix, correlationDepth, testChromosomes, trainingChromosomes, trainingCellType, testCellType, binSize, threads, genomicRegion):

    try:
        cooler_file1 = cooler.Cooler(os.path.join(dataFolder, matrixOutputName))
    except Exception as e:
        traceback.print_exc()
        print(os.path.join(dataFolder, matrixOutputName))
        print(e)
        return None

    try:
        cooler_file2 = cooler.Cooler(originalDataMatrix)
    except Exception as e:
        traceback.print_exc()
        print(originalDataMatrix)
        print(e)
        return None

    os.makedirs(os.path.join(outputFolder, "scores_txt"), exist_ok=True)

    matrixOutputNameWithoutExt = os.path.splitext(matrixOutputName)[0]
    score_dict = {}
    for correlationMethod in correlationMethodList:
        
        for chromosome in testChromosomes:
            if chromosome not in cooler_file1.chromnames:
                return
            if chromosome not in cooler_file2.chromnames:
                return
        
        if correlationMethod == 'pearson_spearman':
            for chrom in testChromosomes:
                score_dataframe = computePearsonCorrelation(pCoolerFile1=os.path.join(dataFolder, matrixOutputName), pCoolerFile2=originalDataMatrix,
                                                            pWindowsize_bp=correlationDepth, pModelChromList=trainingChromosomes, pTargetChromStr=chrom,
                                                            pModelCellLineList=trainingCellType, pTargetCellLineStr=testCellType,
                                                            pPlotOutputFile=None, pCsvOutputFile=None)
                for correlationMethod_ in ['pearson', 'spearman']:
                    for errorType_ in errorType:
                        if correlationMethod_ + '_' + errorType_ in score_dict:
                            score_dict[correlationMethod_ + '_' + errorType_] += score_dataframe.loc[correlationMethod_, errorType_]
                        else:
                            score_dict[correlationMethod_ + '_' + errorType_] = score_dataframe.loc[correlationMethod_, errorType_]

            for correlationMethod_ in ['pearson', 'spearman']:
                for errorType_ in errorType:
                    score_dict[correlationMethod_ + '_' + errorType_] = score_dict[correlationMethod_ + '_' + errorType_] / len(testChromosomes)
            # score = score / len(testChromosomes)

        elif correlationMethod == 'hicrep':
            cool1, binSize1 = readMcool(os.path.join(
                dataFolder, matrixOutputName), -1)
            cool2, binSize2 = readMcool(originalDataMatrix, -1)

            # smoothing window half-size
            h = 5

            # maximal genomic distance to include in the calculation
            dBPMax = 1000000

            # whether to perform down-sampling or not
            # if set True, it will bootstrap the data set # with larger contact counts to
            # the same number of contacts as in the other data set; otherwise, the contact
            # matrices will be normalized by the respective total number of contacts
            bDownSample = False

            # Optionally you can get SCC score from a subset of chromosomes
            sccSub = hicrepSCC(cool1, cool2, h, dBPMax,
                                bDownSample, testChromosomes)

            score_dict[correlationMethod] = np.mean(sccSub)
        elif correlationMethod == 'TAD_score_MSE' or correlationMethod == 'TAD_fraction':
            
            os.makedirs(os.path.join(outputFolder, "tads_predicted_" + matrixOutputNameWithoutExt), exist_ok=True)
            
            chromosomes = ' '.join(testChromosomes)
            arguments_tad = "--matrix {} --minDepth {} --maxDepth {} --step {} --numberOfProcessors {}  \
                            --outPrefix {} --minBoundaryDistance {} \
                            --correctForMultipleTesting fdr --thresholdComparisons 0.5 --chromosomes {}".format(os.path.join(dataFolder,  matrixOutputName), binSize * 3, binSize * 10, binSize, threads,
                            os.path.join(outputFolder, "tads_predicted_" + matrixOutputNameWithoutExt) + '/tads', 100000, chromosomes).split()
            try:
                hicFindTADs.main(arguments_tad)
            except Exception as e:
                traceback.print_exc()
                print(os.path.join(dataFolder, matrixOutputName))
                print(e)
                return

            if correlationMethod == 'TAD_score_MSE':
                tad_score_predicted = os.path.join(
                    outputFolder, "tads_predicted_" + matrixOutputNameWithoutExt) + '/tads_score.bedgraph'
                tad_score_orgininal = os.path.join(
                    tadFolder, "tads_original") + '/tads_score.bedgraph'

                tad_score_predicted_df = pd.read_csv(tad_score_predicted, names=[
                                                        'chromosome', 'start', 'end', 'score'], sep='\t')
                tad_score_orgininal_df = pd.read_csv(tad_score_orgininal, names=[
                                                        'chromosome', 'start', 'end', 'score'], sep='\t')

            
                mean_sum_of_squares = ((tad_score_predicted_df['score'] - tad_score_orgininal_df['score']) ** 2).mean()
                score_dict[correlationMethod] = mean_sum_of_squares
            elif correlationMethod == 'TAD_fraction':
                tad_boundaries_predicted = os.path.join(
                    outputFolder, "tads_predicted_" + matrixOutputNameWithoutExt) + '/tads_boundaries.bed'
                tad_boundaries_orgininal = os.path.join(
                    tadFolder, "tads_original") + '/tads_boundaries.bed'

                tad_boundaries_predicted_df = pd.read_csv(tad_boundaries_predicted, names=[
                                                        'chromosome', 'start', 'end', 'name', 'score', '.'], sep='\t')
                tad_boundaries_orgininal_df = pd.read_csv(tad_boundaries_orgininal, names=[
                                                        'chromosome', 'start', 'end', 'name', 'score', '.'], sep='\t')
                
                tad_fraction = len(tad_boundaries_predicted_df) / len(tad_boundaries_orgininal_df)
                
                exact_matches = pd.merge(tad_boundaries_predicted_df[['chromosome', 'start', 'end']],
                                        tad_boundaries_orgininal_df[['chromosome', 'start', 'end']],
                                        on=['chromosome', 'start', 'end'], how='inner')
                tad_fraction_exact_match = len(exact_matches) / len(tad_boundaries_orgininal_df)

                score_dict[correlationMethod] = tad_fraction
                score_dict[correlationMethod + '_exact_match'] = tad_fraction_exact_match
        elif correlationMethod == 'distribution_fraction':
            pass

    score_dict["hicrep" + '*' + "TAD_fraction" + '*' + "TAD_exact_match_fraction"] = score_dict["hicrep"] * score_dict['TAD_fraction'] * score_dict["TAD_fraction_exact_match"]
    score_dict["(" +"hicrep" + '+' + "TAD_fraction" + '+' + "TAD_exact_match_fraction)" + '/3'] = (score_dict["hicrep"] + score_dict['TAD_fraction'] + score_dict["TAD_fraction_exact_match"]) / 3

    score_dict["TAD_score_MSE" + '*' + "1-TAD_fraction" + '*' + "1-TAD_exact_match_fraction"] = score_dict["TAD_score_MSE"] * score_dict['TAD_fraction'] * score_dict["TAD_fraction_exact_match"]
    score_dict["(TAD_score_MSE" + '+' + "1-TAD_fraction" + '+' + "1-TAD_exact_match_fraction)" + '/3'] = (score_dict["TAD_score_MSE"] + (1- score_dict['TAD_fraction']) + (1-score_dict["TAD_fraction_exact_match"])) / 3

    if genomicRegion:
        score_text = "\n".join([f"{key}: {value:.4f}" for key, value in score_dict.items()])
        print(score_text)
        score_file_path = os.path.join(outputFolder, "scores_txt", matrixOutputNameWithoutExt + "_score_summary.txt")

        with open(score_file_path, 'w') as score_file:
            score_file.write(score_text)
        
        score_text = score_text.replace("\n", "; ")
        browser_tracks_with_hic = """
[hic matrix]
file = {0}
title = {2}
depth = {4}
transform = log1p
file_type = hic_matrix
show_masked_bins = false

[spacer]
height = 0.5

[TAD seperation score]
file = {5}
height = 2
type = lines
individual_color = grey
pos_score_in_bin = center
summary_color = #1f77b4
show_data_range = true
file_type = bedgraph_matrix

[spacer]
height = 1

[hic matrix]
file = {1}
title = original matrix {3}
depth = {4}
transform = log1p
file_type = hic_matrix
show_masked_bins = false
orientation = inverted

[spacer]
height = 0.5

[TAD seperation score]
file = {6}
height = 2
type = lines
individual_color = grey
pos_score_in_bin = center
summary_color = #1f77b4
show_data_range = true
file_type = bedgraph_matrix
    """.format(os.path.join(dataFolder, matrixOutputName), originalDataMatrix, score_text, trainingCellType, 2000000, \
               os.path.join(outputFolder, "tads_predicted_" + matrixOutputNameWithoutExt, 'tads_tad_score.bm'), \
                os.path.join(tadFolder, "tads_original", 'tads_tad_score.bm'))

        tracks_path = os.path.join(
            outputFolder, "browser_tracks_hic.ini")
        with open(tracks_path, 'w') as fh:
            fh.write(browser_tracks_with_hic)

        outfile = os.path.join(
            outputFolder, "pygenometracks", matrixOutputNameWithoutExt + ".pdf")

        arguments = f"--tracks {tracks_path} --region {genomicRegion} "\
                    f"--outFileName {outfile} --trackLabelFraction 0.1 --width 38 --height 35".split()
        try:
            pygenometracks.plotTracks.main(arguments)
        except Exception as e:
            traceback.print_exc()
            print(os.path.join(dataFolder, matrixOutputName))
            print(originalDataMatrix)
            print(e)

    return score_dict

In [ ]:
def createTADTrack(originalDataMatrix, binSize, threads, chromosomes, outputFolder):
    os.makedirs(os.path.join(outputFolder, "tads_original"), exist_ok=True)

    chromosomes_string = 'chr '.join(chromosomes)
    print(chromosomes_string)
    arguments_tad = "--matrix {} --minDepth {} --maxDepth {} --step {} --numberOfProcessors {}  \
                        --outPrefix {} --minBoundaryDistance {} \
                        --correctForMultipleTesting fdr --thresholdComparisons 0.5 --chromosomes {}".format(originalDataMatrix, binSize * 3, binSize * 10, binSize, threads,
                    os.path.join(outputFolder, "tads_original") + '/tads', 100000, chromosomes_string).split()
    print(arguments_tad)
    hicFindTADs.main(arguments_tad)
    tad_score_orgininal = os.path.join(
        outputFolder, "tads_original") + '/tads_score.bedgraph'
    tad_score_orgininal_df = pd.read_csv(tad_score_orgininal, names=[
                                            'chromosome', 'start', 'end', 'score'])

In [ ]:
correlationMethod = ['pearson_spearman', 'hicrep', 'TAD_fraction', 'TAD_score_MSE', 'distribution_fraction']
errorType = ['R2', 'MSE', 'MAE', 'MSLE', 'AUC']
outputFolder = os.path.join(ROOT_PATH_25KB, "result")
dataFolder = ROOT_PATH_25KB_predicted
tadFolder = outputFolder
originalDataMatrix = hic_matrix_25kb_path
correlationDepth = 1000000
testChromosomes = ['1']
trainingChromosomes = ['1']
trainingCellType = "GM12878"
testCellType = "GM12878"
binSize = 25000
threads = 5
genomicRegion = "1:18000000-22000000"

### create the TAD scores

In [ ]:
createTADTrack(hic_matrix_25kb_path, binSize, threads, testChromosomes, outputFolder)

### Score the predict matrices

In [ ]:
max_hicrep_multi = 0
max_hicrep_mean = 0
max_hicrep = 0

min_tad_score_mse_multi = 1
min_tad_score_mse_mean = 1
min_tad_score_mse = 1

max_hicrep_multi_id = ""
max_hicrep_mean_id = ""
max_hicrep_id = ""


min_tad_score_mse_multi_id = ""
min_tad_score_mse_mean_id = ""
min_tad_score_mse_id = ""


scores = {}
scores_none = []
for file in files_25kb:

    score_dict = scoreAndPlot(correlationMethod, errorType, outputFolder, dataFolder, tadFolder, file, originalDataMatrix, correlationDepth, testChromosomes, trainingChromosomes, trainingCellType, testCellType, binSize, threads, genomicRegion)
    if score_dict is None:
        scores_none.append(file)
        continue
    scores[file] = score_dict
    if max_hicrep_multi < score_dict["hicrep" + '*' + "TAD_fraction" + '*' + "TAD_exact_match_fraction"]:
        max_hicrep_multi = score_dict["hicrep" + '*' + "TAD_fraction" + '*' + "TAD_exact_match_fraction"]
        max_hicrep_multi_id = file
    
    if max_hicrep_mean < score_dict["(" +"hicrep" + '+' + "TAD_fraction" + '+' + "TAD_exact_match_fraction)" + '/3'] :
        max_hicrep_mean = score_dict["(" +"hicrep" + '+' + "TAD_fraction" + '+' + "TAD_exact_match_fraction)" + '/3'] 
        max_hicrep_mean_id = file

    if max_hicrep < score_dict["hicrep"]:
        max_hicrep = score_dict["hicrep"]
        max_hicrep_id = file

    
    if min_tad_score_mse_multi > score_dict["TAD_score_MSE" + '*' + "1-TAD_fraction" + '*' + "1-TAD_exact_match_fraction"]:
        min_tad_score_mse_multi = score_dict["TAD_score_MSE" + '*' + "1-TAD_fraction" + '*' + "1-TAD_exact_match_fraction"]
        min_tad_score_mse_multi_id = file
    
    if min_tad_score_mse_mean > score_dict["(TAD_score_MSE" + '+' + "1-TAD_fraction" + '+' + "1-TAD_exact_match_fraction)" + '/3']:
        min_tad_score_mse_mean = score_dict["(TAD_score_MSE" + '+' + "1-TAD_fraction" + '+' + "1-TAD_exact_match_fraction)" + '/3']
        min_tad_score_mse_mean_id = file
    
    if min_tad_score_mse > score_dict["TAD_score_MSE"]:
        min_tad_score_mse = score_dict["TAD_score_MSE"]
        min_tad_score_mse_id = file

    

print("Max hicrep multiplication {} with: {}".format(max_hicrep_multi_id, max_hicrep_multi))
print("Max hicrep mean {} with: {}".format(max_hicrep_mean_id, max_hicrep_mean))
print("Max hicrep {} with: {}".format(max_hicrep_id, max_hicrep))

print("Min TAD score MSE multiplication {} with: {}".format(min_tad_score_mse_multi_id, min_tad_score_mse_multi))
print("Min TAD score MSE mean {} with: {}".format(min_tad_score_mse_mean_id, min_tad_score_mse_mean))
print("Min TAD score MSE {} with: {}".format(min_tad_score_mse_id, min_tad_score_mse))


In [ ]:
scores = {key.replace('.cool', ''): value for key, value in scores.items()}
scores_df = pd.DataFrame.from_dict(scores, orient='index')
scores_df.reset_index(inplace=True)
scores_df.rename(columns={'index': 'File'}, inplace=True)
scores_df.head()

### Load the human ranking file

In [ ]:
ranking_file_path = os.path.join(ROOT_PATH,'image_rankings.csv')
ranking_df = pd.read_csv(ranking_file_path)
print(ranking_df.head())

In [ ]:
ranking_df['Image'] = ranking_df['Image'].str.replace('static/images/', '').str.replace('.jpg', '')
ranking_df

### Merge the scored dataframe with the human ranking

In [ ]:
merged_df = scores_df.merge(ranking_df, left_on='File', right_on='Image')


In [ ]:
merged_df

In [ ]:
merged_df.to_csv(os.path.join(ROOT_PATH, 'merged_df.csv'), index=False)